Here, we add in gasUsedRatio as an exra column to our data. This is a ratio between gas used and gas price, from a free alchemy archive api. The request is limitd to 1024 blocks, so we have sent 10 separate requests in this example to get a total of 10,000 datapoints.

In [15]:
import numpy as np
import pandas as pd
import requests
from web3 import Web3, HTTPProvider
from nltk import flatten

In [5]:
df = pd.read_csv('mev_sample.csv')  # use this to load in most up to date CSV

In [6]:
df2 = (df.set_index('block_number')
   .reindex(range(df.block_number.iat[0],df.block_number.iat[-1]+1), fill_value=0)
   .reset_index())   # get all blocks filled in (including 0 MEV blocks)

In [7]:
df2.head() # ignore 'gas_used' and 'gas_price' columns - this is from old csv

,block_number,miner_reward,gas_used,gas_price,MEV_category
0,11834049,3.720000e+17,4505938,8.261322e+10,3
1,11834050,0.000000e+00,0,0.000000e+00,0
2,11834051,0.000000e+00,0,0.000000e+00,0
3,11834052,0.000000e+00,0,0.000000e+00,0
4,11834053,0.000000e+00,0,0.000000e+00,0


In [2]:
url = "https://mainnet.gateway.pokt.network/v1/lb/62fd2e8c466be4003a7ac327/v1/query/blocktxs" # my free alchemy historical api 

In [3]:
r = requests.post(url) # post request 

In [ ]:
# Initialize a Web3.py instance
web3 = Web3(Web3.HTTPProvider('https://eth-mainnet.g.alchemy.com/v2/PXDcncN7KHOchKhO4S9FzldLoXBtshEI'))

: 

In [8]:
block = 11834049
gas_data = []
for i in range(10):  # adjust for loop based on amount of data required
    hist = web3.eth.fee_history(newest_block = block, block_count=1000)
    gas_data.append(hist['gasUsedRatio'])
    block -= 100

In [16]:
gas_data2 = flatten(gas_data) # change nested list into one dimensional list

In [18]:
len(gas_data2)

10000

In [19]:
df3 = df2.head(10000) # taking 10000 rows of data, so len(gas_data2) = len(df)

In [20]:
df3['gas_used_ratio'] = gas_data2 # adding in gas column

<ipython-input-20-4835273bc893>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['gas_used_ratio'] = gas_data2


In [22]:
df3.drop('gas_price',axis=1,inplace=True) # dont want this column anymore

C:\Users\Toshiba\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [23]:
df3.drop('gas_used',axis=1,inplace=True)  # dont want this column anymore

In [24]:
df3.head() # This is what df should look like

,block_number,miner_reward,MEV_category,gas_used_ratio
0,11834049,3.720000e+17,3,0.999796
1,11834050,0.000000e+00,0,0.999850
2,11834051,0.000000e+00,0,0.999542
3,11834052,0.000000e+00,0,0.998738
4,11834053,0.000000e+00,0,0.998839
